<img src="../images/AzPTravel_PPM.png">

# Audit Report

## This script reads the output of the of the final data files and compares them to inputs. 

#### Euro Check
- Compare the deltas on teh Localcurr to the source and raise errors if they do not match

#### Prior over Period Check
- Quickly compare period over period and YoY final data files for major KPI's and row counts. 



### User Variables
- These are overwritten if inherited from run_control.ipynb.
- Feel Free to reset them for a manual run if you like
- Do not save without percode = "-f"

In [1]:
commit_message = "Development and testing."
# Give a brief reason for the run.

run_control = 1
#run_type = 0 - Lite run with no reporting, not recommended.
#run_type = 1 - Lite run with normal reporting, default setting.
#run_type = 2 - Heavy run with full reporting, available for audits and troubleshooting.
#run_type = 5 - A default setting. Indicates the script is being run by an outside process without an inherited value

percode = "2021.Q1"
# Data Collection Code, this controls file paths and output names
# "-f" is the value indicating a bad inheritance from run with arg

s_format = "p"
# denotes the source data format x == Excel; j == json, p == parquet

#----------
# do not edit - this either inherits the full instance timestamp from the papermill book or captures the run time of this script.
from datetime import datetime  # datetime options
inst_datetime = datetime.now().strftime("%m%d%Y%H%M%S")

In [2]:
# Parameters
run_control = 1
percode = "2021.Q1"
commit_message = "Live run, added AT after conversion from Excel"
inst_datetime = "06012021170006"


#### Notebook display options

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


#### import packages

In [4]:
#### Packages used

import os # System commands
import sys # System commands

import warnings # custom warnigns options

import glob # Directory operations
import getpass # Work with parquet
import json # Work with json

import matplotlib.pyplot as plt #Plots and Graphs
import seaborn as sns #Plots and Graphs
import numpy as np # Series and math
import pandas as pd #DataFrame and math

#excel operations
import re
import nicexcel as nl # Excel operations
import xlsxwriter # Excel operations
import openpyxl # Excel operations

#Not sure, test if needed
import timeit



#### Default Variables, these govern logic, do not edit.

In [5]:
default_dc = "20XX.QX"
default_rc = 0 #extra lite mode
dummy_perc = "33Q3" # bad inheritance

#### Script determining run context ie, manual, run_control.ipynb, or other.

In [6]:
if run_control == 5:
    run_control = default_rc 
else:
    run_control = run_control

try:
    if sys.argv[1] == "-f":
        percode = percode
    else:
        percode = sys.argv[1]

except IndexError:
    percode = default_dc
except NameError:
    percode = default_dc


if percode == "2021.Q1":
    
     prev_percode = "2020.Q4"
     prev_yearcode = "2020.Q1"

elif percode == "2020.Q4":
    
     prev_percode = "2020.Q3"
     prev_yearcode = "2019.Q4"

elif percode == "2020.Q3":
    
     prev_percode = "2020.Q1"
     prev_yearcode = "none"    

elif percode == "2020.Q1":
    
     prev_percode = "2019.Q4"
     prev_yearcode = "none"   

#### style settings

In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.float_format', lambda x: '%.5f' % x)


## Begin Input Read
- Make paths used in script
- all paths should be modular and easily replaced ahead of operations.

In [8]:
#root directory
rt_path = f'\\\\hecate\\Insurance_US\\Product Development\\Product Management\\Global PPM\\Reporting\Data Collection'

# Euro file dir
euro_conv_source_path =current_path = os.path.join(rt_path ,percode,'External Resources')

# Source file dirs
current_path = os.path.join(rt_path ,"Production" , percode)
previous_period_path = os.path.join(rt_path ,"Production" , prev_percode)
previous_year_path = os.path.join(rt_path ,"Production" , prev_yearcode )




#### Make sources used in script
- all sources should be modular and easily replaced ahead of operations.

In [9]:
if "Q1" in percode:

    EURRATES = str.replace(f"EURRATES {percode}.xlsx","Q1","03.31")

elif  "Q2" in percode:

    EURRATES = str.replace(f"EURRATES {percode}.xlsx","Q2","06.30")

elif  "Q3" in percode:

    EURRATES = str.replace(f"EURRATES {percode}.xlsx","Q2","09.30")
    
elif  "Q4" in percode:

    EURRATES = str.replace(f"EURRATES {percode}.xlsx","Q2","12.31")
    
eurrates = os.path.join( euro_conv_source_path,   EURRATES)



current_lc_file = os.path.join(current_path  ,f'{percode}.localcur.csv')
current_eur_file  = os.path.join(current_path  ,f'{percode}.euroconv.csv')

previous_period_lc_file = os.path.join(previous_period_path ,f'{ prev_percode}.localcur.csv')
previous_period_eur_file  = os.path.join(previous_period_path ,f'{ prev_percode}.euroconv.csv')

previous_year_lc_file = os.path.join(previous_year_path  ,f'{ prev_yearcode}.localcur.csv')
previous_year_eur_file  = os.path.join(previous_year_path ,f'{ prev_yearcode}.euroconv.csv')


#### import data

In [10]:
eurrates_df = pd.read_excel(eurrates,sheet_name = "Output")


current_lc_df = pd.read_csv(current_lc_file , engine='python') 
current_eur_df = pd.read_csv(current_eur_file , engine='python') 


previous_period_lc_df = pd.read_csv(previous_period_lc_file, engine='python')
previous_period_eur_df = pd.read_csv(previous_period_eur_file , engine='python')


previous_year_lc_df = pd.read_csv(previous_year_lc_file, engine='python') 
previous_year_eur_df = pd.read_csv(previous_year_eur_file , engine='python')



put_cell 21 36


KeyboardInterrupt: 

## Begin Audits

#### Euro Conversion Check.
- Each major KPI evaluated bu Country by Currency in the source files and the official Allianz Connect reference report.

In [ ]:
attributes = ['country', 'currency']

alt_attributes = ['Country', 'Currency']

comparison_fields = [
 'written_revenues_net_of_taxes',
 'written_revenues',
 'earned_revenues_net_of_taxes',
 'earned_revenues',
 'total_compensation',
  'actual_incurred_losses_(paid_+_ocr_+_ibnr)',
 'total_expenses',
 'profit_or_loss',
]


alt_comparison_fields = [
'Written Revenues net of Taxes',
 'Written Revenues',
 'Earned Revenues net of Taxes',
 'Earned Revenues',
 'Total Compensation',
 'Actual Incurred Losses (Paid + OCR + IBNR)',
 'Total Expenses',
 'Profit or Loss',
]

view_cols = [] 

for a in attributes: view_cols.append(a)
for c in comparison_fields: view_cols.append(c)

    
    
alt_view_cols = [] 

for a in alt_attributes: alt_view_cols.append(a)
for c in alt_comparison_fields: alt_view_cols.append(c)
    

current_eur_view =  current_eur_df[view_cols].groupby(attributes).sum()

try:
    current_lc_view =  current_lc_df[view_cols].groupby(attributes).sum()

except KeyError:
    current_lc_view =  current_lc_df[alt_view_cols].groupby(alt_attributes).sum()  
    

previous_period_eur_view =  previous_period_eur_df[view_cols].groupby(attributes).sum()    
previous_year_eur_view =  previous_year_eur_df[view_cols].groupby(attributes).sum()    




In [ ]:

calculated_rates = current_lc_view.div(current_eur_view).reset_index()

calculated_rates = calculated_rates.melt(id_vars=attributes).reset_index(drop = True)

calculated_rates = calculated_rates[["country","currency", "value"]]

calculated_rates['value'] = calculated_rates['value'].round(5)

calculated_rates = calculated_rates.drop_duplicates()[calculated_rates['value'].notna()]
calculated_rates 

In [ ]:
test = 1

if calculated_rates.groupby( attributes).count()['value'].max() != 1: 
    
    test = 0

else:
    
    pass
        
        
1/test # if this is an error you have bad currency conversions and possibly a bad join with extra rows

#todo build a proper error class

In [ ]:

test = 1

if len(current_eur_df[attributes].drop_duplicates()) != len(calculated_rates): 
    
    test = 0

else:
    pass

1/test # if this is an error you do not have the same number of country and curreny combos in all views. Major problem!

#todo build a proper error class



In [ ]:
conversion_comparison =  calculated_rates.merge( eurrates_df,  how='left',  left_on='currency', right_on='CurrCode' )

conversion_comparison




In [ ]:
euro_df = conversion_comparison

%store euro_df


In [ ]:

test = 1

if (   conversion_comparison['value']   - conversion_comparison['EUR_2_LC']).sum() != 0: 
    
    test = 0

else:
    pass

1/test # if this is an error you do not match the source currency document. Major problem!

#todo build a proper error class


#### Data views for evaluation
- Create several views in order to populate graphs

In [ ]:
current_eur_view['Period'] = percode
previous_period_eur_view['Period']  =    prev_percode 
previous_year_eur_view['Period']    =  prev_yearcode 



In [ ]:
rowcounts = pd.DataFrame()

rowcounts_current = current_eur_df[["country","written_revenues_net_of_taxes"]].groupby("country").count() 

rowcounts_current["Period"] = percode


rowcounts_prev_per = previous_period_eur_df[["country","written_revenues_net_of_taxes"]].groupby("country").count() 

rowcounts_prev_per["Period"] = prev_percode 


rowcounts_py = previous_year_eur_df[["country","written_revenues_net_of_taxes"]].groupby("country").count() 

rowcounts_py["Period"] = prev_yearcode 


rowcounts = rowcounts.append(rowcounts_current).append(rowcounts_prev_per).append(rowcounts_py)
rowcounts = rowcounts.rename( { 'written_revenues_net_of_taxes' :'Rowcounts' }, axis='columns')

rowcounts = rowcounts.reset_index()




In [ ]:
combo_views = current_eur_view.append(previous_period_eur_view).append(previous_year_eur_view)

combo_views

In [ ]:
combo_views = combo_views.reset_index()

melt_cv = combo_views.melt(['country','Period','currency'], ignore_index = True)

melt_cv['value'] = melt_cv['value'].fillna(0)


In [ ]:
check_set0 = melt_cv[melt_cv["Period"] == percode].groupby( ["country"]).sum()

include = check_set0.index.values

include = list(include)

melt_cv = melt_cv[(melt_cv["country"].isin(include)) & (~melt_cv["variable"].isin(["index","written_revenues","earned_revenues"]) )]




#Rowcount visual
- see how many rows were submitted by each BU in this data submission vs the previous submission and the YoY submission.

In [ ]:
order = rowcounts.Period.unique()

included = (include * 6)
included.sort()

g = sns.FacetGrid(rowcounts, col="country",  hue="Period", palette="Set3",col_wrap = 4, height=2, aspect=2, margin_titles=True, legend_out=False, sharex=False, sharey=False)
g.map(sns.barplot, "Period", "Rowcounts", order=order, ci=None, label = True)
axes = g.axes.flatten()
for ax, b  in zip(axes, include):
    ax.set_xlabel(b)
    ax.set_ylabel("Rows Submitted")


g.savefig('rows.png', dpi=125)

#KPI visual
- Displays in KPI totals in Euros for this data submission vs the previous submission and the YoY submission.

In [ ]:
included = (include * 6)
included.sort()

g = sns.FacetGrid(melt_cv, row="country", col="variable",  hue="Period", palette="Set3", height=2, aspect=2, margin_titles=True, legend_out=False, sharex=False, sharey=False)
g.map(sns.barplot, "Period", "value", order=order, ci=None, label = True)
axes = g.axes.flatten()
for ax, b  in zip(axes, included):
    ax.set_xlabel(b)
    ax.set_ylabel("Euro")
    
g.savefig('KPIs.png', dpi=125)

In [ ]:
order = melt_cv.variable.unique()


included = (include * 6)
included.sort()

g = sns.FacetGrid(melt_cv, row="country", col= "Period" , hue= "variable",  palette="Set3", height=2, aspect=2, margin_titles=True, legend_out=False, sharex=False, sharey=False)
g.map(sns.barplot, "variable", "value", order=order, ci=None, label = True)
axes = g.axes.flatten()
for ax, b  in zip(axes, included):
    ax.set_xlabel(b)
    ax.set_ylabel("Euro")
    
g.savefig('KPIs_vw2.png', dpi=200)